In [3]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms

class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    inception_model = models.inception_v3(pretrained=True)
    inception_model.fc = nn.Linear(2048, 64)
    self.feature_extractor = inception_model

  def forward(self, x):
    x = transforms.functional.resize(x,size=[224, 224])
    x = x/255.0
    x = transforms.functional.normalize(x, 
                                            mean=[0.485, 0.456, 0.406], 
                                            std=[0.229, 0.224, 0.225])
    return self.feature_extractor(x).logits

model = MyModel()
model.eval()
saved_model = torch.jit.script(model)
saved_model.save('../input/google-img-embed/inception_v3_saved_model.pt')

##### [.logits] work only with torchscripts

In [1]:
from PIL import Image
import torch
from torchvision import transforms

# Model loading.
model = torch.jit.load('../input/google-img-embed/resnet50_saved_model.pt')
model.eval()
embedding_fn = model

# Load image and extract its embedding.
input_image = Image.open('../input/images/dogs/collie-beach-bokeh.jpg').convert("RGB")
convert_to_tensor = transforms.Compose([transforms.PILToTensor()])
input_tensor = convert_to_tensor(input_image)
input_batch = input_tensor.unsqueeze(0)
with torch.no_grad():
  embedding = torch.flatten(embedding_fn(input_batch)[0]).cpu().data.numpy()
  print(embedding)

[0.49913645 0.46906117 0.49825194 0.37195235 0.45691285 0.39204645
 0.4079221  0.4142578  0.4886418  0.5636962  0.39359304 0.49995944
 0.6120907  0.39657354 0.48164067 0.35197148 0.48297414 0.4926043
 0.5482643  0.49669454 0.43279284 0.53634715 0.5341487  0.52608174
 0.3791866  0.46428728 0.46451157 0.54932964 0.40856847 0.6544852
 0.47272003 0.4910925  0.46992144 0.4340453  0.47640014 0.56555897
 0.36480945 0.4114953  0.58823776 0.35343066 0.43007874 0.50794774
 0.3404074  0.43273982 0.48107296 0.4425872  0.64142925 0.4329255
 0.5477964  0.42309114 0.5992905  0.4794149  0.51749676 0.5859948
 0.4350825  0.46696788 0.46880722 0.31757396 0.4422681  0.38940272
 0.45007625 0.34040406 0.49756482 0.40075022]
